In [1]:
import pandas as pd
import sys, os

sys.path.append(os.path.abspath(".."))

# Load data
financials = pd.read_csv("../data/cleaned/financial_statements.csv")

# Imports
from engines.composite_risk_engine import composite_risk_engine
from engines.ratio_engine_core import ratio_engine
from engines.trend_engine import trend_engine
from engines.cash_flow_engine import cash_flow_engine
from engines.anomaly_efficiency_engine import anomaly_efficiency_engine
from engines.solvency_engine import solvency_engine


In [2]:
from config.defaults import DEFAULT_CLIENT_CONFIG

config = DEFAULT_CLIENT_CONFIG


In [3]:
ratio_results = ratio_engine(financials)

ratios_df = pd.DataFrame(ratio_results)
ratios_df.sort_values(["Company", "Year"], inplace=True)
ratios_df.reset_index(drop=True, inplace=True)
trend_results = trend_engine(ratios_df)

cash_flow_results = cash_flow_engine(financials)
anomaly_results = anomaly_efficiency_engine(ratios_df)
solvency_results = solvency_engine(ratios_df)

✅ trend_engine output validated successfully.
✅ cash_flow_engine output validated successfully.
✅ anomaly_efficiency_engine output validated successfully.
✅ solvency_engine output validated successfully.


In [4]:
composite_results = composite_risk_engine(
    trend_results,
    cash_flow_results,
    anomaly_results,
    solvency_results,
    config
)

composite_results


[{'Company': 'Acme Manufacturing Ltd',
  'Year': 2023,
  'composite_score': 0,
  'risk_band': 'low'},
 {'Company': 'Banyan Retail Co',
  'Year': 2023,
  'composite_score': 20,
  'risk_band': 'low'},
 {'Company': 'Coastal Tech Ltd',
  'Year': 2023,
  'composite_score': 20,
  'risk_band': 'low'},
 {'Company': 'Emerald Energy Inc',
  'Year': 2023,
  'composite_score': 0,
  'risk_band': 'low'}]

In [5]:
def inspect_keys(results, name):
    print(f"\n{name} sample keys:")
    for r in results[:3]:
        print(r.keys())

inspect_keys(trend_results, "trend")
inspect_keys(cash_flow_results, "cash")
inspect_keys(anomaly_results, "anomaly")
inspect_keys(solvency_results, "solvency")



trend sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])

cash sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])

anomaly sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])

solvency sample keys:
dict_keys(['engine', 'Company', 'Year', 'metrics', 'flags', 'severity', 'explanation'])
dict_keys(['engine', 'Company', 'Yea

In [6]:
pd.DataFrame(composite_results)

,Company,Year,composite_score,risk_band
0,Acme Manufacturing Ltd,2023,0,low
1,Banyan Retail Co,2023,20,low
2,Coastal Tech Ltd,2023,20,low
3,Emerald Energy Inc,2023,0,low


In [7]:
# build cash index
c = {(r["Company"], r["Year"]): r for r in cash_flow_results}

# now you can inspect cash contribution
for r in composite_results:
    key = (r["Company"], r["Year"])
    cash = c.get(key)
    print(r["Company"], r["Year"], "Cash Severity:", cash.get("severity") if cash else "N/A")


Acme Manufacturing Ltd 2023 Cash Severity: stable
Banyan Retail Co 2023 Cash Severity: action
Coastal Tech Ltd 2023 Cash Severity: action
Emerald Energy Inc 2023 Cash Severity: stable
